# Pandas quick start
Pandas is a 3rd-party, open source, library used to for data science. It is perhaps the most important library for you, as a student of analytics.

At a high level, Pandas provides the following functionality:
1. Reading and writing data in various formats: csv, sql, feather and many others
2. A set of data structures in which to store data (so higher level than lists, tuples and dictionaries)
3. Functions to transform data in _many_ ways: individual columns, operating on multiple columns at once, aggregating in total or in by categories (aka group by), visualizing datasets, etc.

Further, _upstream_ libraries, such as ones providing machine learning algorithms (scikit-learn) know how to consume Pandas data structures.

Extremely helpful Pandas cheatsheet: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf (search the web, there are many more, just as useful)

In [ ]:
import numpy as np # <= numpy is only used once, for the np.log function (although pandas is built on top of it)
import pandas as pd # <= `pd` is almost always the abbreviation used for pandas

import seaborn as sns # <= seaborn is not part of pandas, but very useful charting library (built on top of matplotlib)
from bokeh.plotting import figure, show, output_file # <= another charting library, can be more interactive
from bokeh.io import output_notebook

In [ ]:
# remember, since we will be drawing some charts, we need to execute this line - because seaborn uses matplotlib
%matplotlib inline 

# the following line tells pandas to avoid scietific notation
pd.set_option('display.float_format', '{:.2f}'.format)

# bokeh code to make it work in jupyter notebooks
output_notebook()

In [ ]:
%reload_ext postcell
%postcell register

## Quick walk-through of Pandas

### Load file and take a quick look at it

Note that this file is available at: https://www.kaggle.com/kumarajarshi/life-expectancy-who/home
Go to that URL, click 'Download' which will start downloading a zip file. Load it as shown below:

In [ ]:
# Read csv file
life_df = pd.read_csv("../../datasets/life-expectancy/life-expectancy-who.zip2", compression='zip')

In [ ]:
life_df.head() # Look at the first 5 lines to visually inspect data

In [ ]:
life_df.shape # This file has 2,938 records (rows) and 11 columns

In [ ]:
life_df.columns # List of columns

**WARNING** Notice that some columns have an extra space!

In [ ]:
life_df.dtypes

In [ ]:
life_df.describe() # quick summary of all the columns

In [ ]:
life_df.describe().T # Flip or `transpose` the data for easier viewing

### Visualize data

In [ ]:
# Warning, this step may take a minute or two to complete
%time sns.pairplot(life_df) # look at all variables at once - pair-plot

The previous plot isn't very useful because there are too many columns. What if we had fewer columns? Let's just select the first few columns:

In [ ]:
first_few_df = life_df[['Country', 'Year', 'Population', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ']]
first_few_df.head()

### Selecting columns

Given dataframe `df`, you select columns by passing it a list of columns:

```python
list_of_cols = ['col1', 'col2', 'col3']
df[list_of_cols]
```

or

```python
df[['col1', 'col2', 'col3']] # <= notice two square brackets!
```

**Exercise** Pick columns 'Country', 'Year' and 'Population' from dataframe 'life_df'

In [ ]:
%%postcell exercise_030_a

#type your answer here

Let's also limit the data to the year 2015

In [ ]:
first_few_2015_df = first_few_df[first_few_df.Year == 2015]
first_few_2015_df

In [ ]:
first_few_2015_df.shape

### Selecting rows
Given dataframe `df`, a criteria can be given as an expression which returns `True` for every row which should be selected

```python
mask = df.col1 < 5
df[mask]
```
or

```python
df[df.col1 < 5]
```

**Exercise** Show rows in 'life_df' for Afghanistan

In [ ]:
%%postcell exercise_030_b

#type your answer here

#### Modern Pandas suggests using `.loc[]` to select rows and columns

In [ ]:
life_df.loc[life_df.Year == 2015, ['Country']].shape

In [ ]:
%time sns.pairplot(first_few_2015_df)

In [ ]:
first_few_2015_df['Population'] # Show the column 'Population'

In [ ]:
first_few_2015_df['Life expectancy '] # <= Notice the extra space!

#### Visualize the relationship between 'Population' and 'Life expectancy '

In [ ]:
sns.scatterplot(x='Population', y='Life expectancy ', data=first_few_2015_df)

In [ ]:
sns.scatterplot(x='Population', y='Life expectancy ', data=first_few_2015_df[first_few_2015_df.Population < 10_500_000])

In [ ]:
TOOLTIPS = [('Country', '@Country')]

p = figure(title = "Pop vs Life expectancy", tooltips=TOOLTIPS)

p.circle('Population', 'Life expectancy ', source=first_few_2015_df,fill_alpha=0.2, size=10)
show(p)

In [ ]:
first_few_2015_df['Life expectancy '].value_counts() # So what are common life expectancies?

In [ ]:
min(first_few_2015_df['Life expectancy ']), max(first_few_2015_df['Life expectancy '])

In [ ]:
min(first_few_2015_df['Population']), max(first_few_2015_df['Population'])

In [ ]:
%timeit min(first_few_2015_df['Population']), max(first_few_2015_df['Population'])
%timeit np.min(first_few_2015_df['Population']), np.max(first_few_2015_df['Population'])
%timeit first_few_2015_df['Population'].min(), first_few_2015_df['Population'].max()

Previous list of numbers is not very useful, let's plot the distribution:

In [ ]:
first_few_2015_df['Life expectancy '].plot.hist()

### Investigate infant deaths

In [ ]:
first_few_2015_df['infant deaths'].plot.hist()

In [ ]:
np.log(first_few_2015_df['infant deaths']+1).plot.hist() # just to "zoom" in quickly - dirty hack

In [ ]:
#There are some cases where infant deaths are over 200???
first_few_2015_df[first_few_2015_df['infant deaths'] > 200]

The numbers for infant deaths are _so_ high that we need to go back to the data source and double check our understanding.

**Exercise** Check the exact definition of 'infant deaths.'

The 'Measles' value is defined as 'number of reported cases per 1,000 population.' Let's find the actual number of measles per country (in 2015):

In [ ]:
first_few_2015_df['Measles'] * first_few_2015_df['Population'] # What happened? (hint, extra space)
# Why did you get the error and have you seen that error before? 

In [ ]:
# In the calculation below, notice that we just added the two vectors, as if they were numbers...no loops!!
first_few_2015_df['Measles '] * first_few_2015_df['Population']

Something _very_ interesting happened above. We added two lists or vectors of numbers, without using a loop! Pandas and numpy (and matrix math) works this way.

Let's add this column back to our data frame:

In [ ]:
# We are creating a new column!
first_few_2015_df.loc[:, 'Total Measles'] = first_few_2015_df['Measles '] * first_few_2015_df['Population']

What's with that warning?
Turns out, `first_few_2015_df` is not the original dataframe. It is a subsetted view. If we update this subset, what happens to the remaining rows and columns? Better to modify the original dataframe.

In [ ]:
life_df.loc[:, 'Total Measles'] = life_df['Measles '] * life_df['Population']

In [ ]:
first_few_2015_df

### Cheap version of Pandas

Let's try to make a very tiny, very silly version of Pands ourselves.

The first implementation is just a dictionary with column names as keys and lists of data as values. Here is an example:

In [ ]:
import random

In [ ]:
df = {
   "col1": [random.random() for x in range(10)]
 , "col2": [random.random() for x in range(10)]
 , "col3": [random.random() for x in range(10)]
}
df

In [ ]:
df['col1']

#### Write a function to read csv files

In [ ]:
import collections

def create_df_from_csv(file):
    
    num_of_columns = None
    header = None
    df = collections.defaultdict(list)
    
    with open(file, "r") as f:
        for line in f:
            tokens = line.split(",") # recall that this is not the best way to parse csv files (python has a csv library built-in)
            if not num_of_columns: 
                num_of_columns = len(tokens) # count the number of columns in the first row
                header = [t.strip() for t in tokens] # assumes the first row will always contain header
            else:
                for idx, col in enumerate(header): df[col].append(tokens[idx].strip()) # assumes all rows have equal number of columns
    return df

For the next step, create a 10 line version of the file in datasets/life-expectancy, otherwise there will be too much data for you to see the structure of the dataframe (but the code should work, either way)

In [ ]:
le_df = create_df_from_csv("../../datasets/life-expectancy/life_expectancy_10.csv")
le_df

In [ ]:
le_df['Schooling']

**Exercise** The function above is not _production ready._ What are some ways things can go wrong?
**Exercise** How will classes combine the function (verb) and the dataframe data structure (noun)?